In [24]:
import pandas as pd
import os
import numpy as np

path = 'C:\\Users\\Mateusz Jodlowski\\Desktop\\mgr\\t-sne'
os.chdir(path)

movies = pd.read_csv('tmdb-movies.csv', encoding = 'utf8')
movies.dropna(subset=['overview'], inplace=True)
movies.apply(lambda x: pd.api.types.infer_dtype(x.values))


genres = movies['genres'].str.split('|',expand=True)
genres = genres.values
genres = pd.DataFrame(genres)
genres.columns = ['Genre 1', 'Genre 2', 'Genre 3', 'Genre 4', 'Genre 5']
genres = genres.applymap(lambda x: '' if x is None else str(x))

# Only 1 genre
movies = movies.iloc[(genres.apply(lambda x: sum(x != ''), axis = 1) == 1).values]
movies = movies.rename(columns = {'genres':'Genre 1', 'overview':'Description'})

# More than 1 sentence
movies = movies.iloc[((~movies['Description'].str.split('\\. ',expand=True).isnull()).apply(sum, axis = 1) > 2).values]

# Both genre and description not null
movies = movies.iloc[((movies[['Genre 1', 'Description']].isnull()).apply(sum, axis = 1) == 0).values]

# Genre share at least 10%
movies = movies.set_index('Genre 1').join(pd.DataFrame(movies.groupby('Genre 1').size()/len(movies) > 0.1)).rename(columns = {0:'to_drop'}).reset_index()

movies = movies.iloc[movies['to_drop'].values]
movies = movies.drop(['to_drop'], axis = 1)

from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer

stop = stopwords.words('english')

docs = movies['Description']

docs = docs.apply(lambda x: x.translate(str.maketrans({key: None for key in string.punctuation})))
docs = docs.apply(lambda x: x.lower())

docs = docs.apply(lambda x: [x for x in x.split() if x not in stop])
docs = docs.apply(lambda x: ' '.join(x))

lancaster = LancasterStemmer()

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# docs = [lancaster.stem(word) for word in docs]

docs = [wordnet_lemmatizer.lemmatize(word) for word in docs]

vec = CountVectorizer()

X = vec.fit_transform(docs)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

# Words occurred in more than 1 movie
df = df.iloc[:,((df>0).apply(sum) > 1).values]
import re
df = df.iloc[:, pd.Series(df.columns).apply(lambda x: re.match("^[0-9]", x) is None).values]
df



,aaron,abandoned,abby,abilities,ability,able,aboard,abroad,absolute,absolutely,...,youth,youthful,youâ,zac,zana,zero,zombie,zone,œi,œthe
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
dtm = movies['Genre 1'].reset_index()[movies['Genre 1'].reset_index().columns.difference(['index'])]
groups_filter = pd.concat([dtm, df], axis=1)
groups_filter = groups_filter.groupby('Genre 1').sum()[df.columns]
groups_filter = groups_filter.T
groups_filter['n_words'] = groups_filter.apply(sum, axis = 1)

# Taking into consideration n_words which is always > 0
groups_filter['is_common'] = groups_filter.apply(lambda x: 1 if sum(x > 0) > 2 else 0, axis = 1)

groups_filter.sort_values(['is_common', 'n_words'], ascending = False)

# Get rid of top 20% words which are common (?)
groups_filter = groups_filter.sort_values(['is_common', 'n_words'], ascending = False).iloc[int(np.floor(groups_filter.shape[0]/5)):groups_filter.shape[0]]

word_list = list(groups_filter.index)
word_list

['appears',
 'apple',
 'audience',
 'avoid',
 'bar',
 'beth',
 'biggest',
 'billy',
 'bond',
 'building',
 'captured',
 'care',
 'cast',
 'center',
 'character',
 'choice',
 'choose',
 'christian',
 'clients',
 'close',
 'competition',
 'connect',
 'constant',
 'cops',
 'count',
 'courage',
 'court',
 'creative',
 'deeply',
 'derek',
 'dinner',
 'discovery',
 'disorder',
 'dramatic',
 'due',
 'earth',
 'eccentric',
 'eight',
 'ellis',
 'emotional',
 'eventually',
 'except',
 'experiment',
 'exploration',
 'explosive',
 'eye',
 'fame',
 'fears',
 'feelings',
 'filming',
 'flight',
 'follow',
 'fresh',
 'generation',
 'gone',
 'government',
 'half',
 'halloween',
 'haunted',
 'health',
 'henry',
 'hero',
 'holiday',
 'human',
 'ice',
 'illness',
 'institution',
 'interview',
 'involving',
 'issues',
 'jason',
 'jay',
 'jesus',
 'jim',
 'join',
 'josh',
 'kill',
 'legal',
 'less',
 'let',
 'loves',
 'machine',
 'manage',
 'marry',
 'marsh',
 'mechanic',
 'media',
 'medical',
 'mental',
 '

In [56]:
groups_filter.iloc[:, 0:groups_filter.shape[1]-2]
groups_filter = groups_filter.iloc[:, 0:groups_filter.shape[1]-2]
n_filter = (groups_filter > 0).sum()
n_filter

Genre 1
Comedy         2142
Documentary    1475
Drama          2263
Horror         1084
dtype: int64

In [57]:
groups_filter = groups_filter.T

def sorted(s, num):
    tmp = s.sort_values(ascending=False)[:num]  # earlier s.order(..)
#     tmp.index = range(num)
    return tmp

groups_filter = groups_filter[word_list]
groups_filter
sorted_words = groups_filter.T.apply(lambda x: sorted(x, 500))
sorted_words

Genre 1,Comedy,Documentary,Drama,Horror
abby,NaN,NaN,4.0,NaN
abilities,NaN,NaN,NaN,3.0
abuse,NaN,NaN,NaN,2.0
abusive,NaN,NaN,4.0,NaN
accounted,NaN,2.0,NaN,NaN
accounts,NaN,2.0,NaN,NaN
ace,4.0,NaN,NaN,NaN
acting,NaN,NaN,4.0,NaN
activism,NaN,3.0,NaN,NaN
adams,NaN,NaN,4.0,NaN


In [286]:
# Each row now has 4 intersections with all 4 genres
df_array = df[list(sorted_words.index)].values
tfm = [row * (~sorted_words.isnull().T) for row in df_array]

[             abby  abilities  abuse  abusive  accounted  accounts  ace  \
 Genre 1                                                                  
 Comedy          0          0      0        0          0         0    0   
 Documentary     0          0      0        0          0         0    0   
 Drama           0          0      0        0          0         0    0   
 Horror          0          0      0        0          0         0    0   
 
              acting  activism  adams ...  yearold  youll  youngest  youth  \
 Genre 1                              ...                                    
 Comedy            0         0      0 ...        0      0         0      0   
 Documentary       0         0      0 ...        0      0         0      0   
 Drama             0         0      0 ...        0      0         0      0   
 Horror            0         0      0 ...        0      0         0      0   
 
              youthful  zac  zana  zombie  zone  œi  
 Genre 1                

In [ ]:

df = df.iloc[:, (groups_filter.apply(lambda x: sum(x != 0)) == 1).values]

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(df)
tfidf = pd.DataFrame(tfidf.toarray())
tfidf.columns = df.columns.difference(['Genre 1'])
tfidf

tfidf = tfidf.reset_index()[tfidf.reset_index().columns.difference(['index'])]
first_class = pd.concat([dtm, tfidf], axis=1)
first_class